Import Libraries

In [295]:
#%pip install filterpy


In [1]:
from ultralytics import YOLO
import utils
import cv2
from sort.sort import *
import numpy as np
from utils import Get_Car, Read_License_Plate, Write_CSV    
import pandas as pd

Using CPU. Note: This module is much faster with a GPU.


#Load Models


In [297]:
#cocomodels trained on coco dataset to detect vehicles
model = YOLO('yolov8n.pt')
LP_detector = YOLO('license_plate_detector.pt')

Load Video

In [298]:
cap =cv2.VideoCapture('testUK3.mp4')
vehicles = [2,3,5,7]


In [301]:

utils.Read_License_Plate()

TypeError: Read_License_Plate() missing 1 required positional argument: 'text'

Read Frames

In [ ]:



frame_num =-1
ret = True
mot_tracker = Sort()
results = {}
while ret:
    frame_num += 1
    f = True
    ret, frame = cap.read()

    if ret and frame_num < 15:
        results[frame_num]= {}
 #detect Vehicles
        detections = model(frame)[0]
        detections_ = []
       # print(detections)
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            #print(detection)
            if int(class_id) in vehicles:
                detections_.append([x1,y1,x2,y2, score])

#track vehicles from videos
        
        track_ids = mot_tracker.update(np.asarray(detections_))
        print("result track ids:", track_ids)
#Detect License Plates
        LP = LP_detector(frame)[0]
        for lp in LP.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = lp
            print("License Plate Detection:", lp)

        #assign license plate to vehicle
            if track_ids.any():
                xcar1, ycar1, xcar2, ycar2, car_id = Get_Car(lp,track_ids)

            #crop License plate
                lp_crop = frame[int(y1): int(y2), int(x1):int(x2),:]

            #process License plate
                    
                lp_crop_gray = cv2.cvtColor(lp_crop, cv2.COLOR_BGR2GRAY)
                _, lp_crop_tsh = cv2.threshold(lp_crop_gray,95 , 255, cv2.THRESH_BINARY_INV )
                # cv2.imshow('lp_crop', lp_crop)
                # cv2.imshow('Threshold', lp_crop_tsh)
                # cv2.waitKey(0)
            
            # Read License Plate text
                lp_text, lp_text_score = Read_License_Plate()

                if lp_text is not None:
                    #car_id = str(car_id.tolist()) 
                    results[frame_num][str(car_id)] = {
                        'car' : {'bbox':[xcar1, ycar1, xcar2, ycar2]},
                        'license_plate': {
                            'bbox': [x1, y1, x2, y2], 
                            'text': lp_text,
                            'bbox_score':score,
                            'text_score':lp_text_score
                                                }
                                            }                                                                
                                                                
                
                    print('results_final', results[frame_num][str(car_id)])
                                                
                

            #Write results
Write_CSV(results, 'results.csv')


                   


                


        
        

                        


                
        




0: 384x640 2 cars, 1 truck, 312.0ms
Speed: 13.0ms preprocess, 312.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
result track ids: [[      450.4     0.68884         768      204.65         241]
 [      196.3      1.5921      460.86       179.5         240]
 [     450.87     0.62168       767.4      204.12         239]]

0: 384x640 3 license_plates, 266.0ms
Speed: 7.0ms preprocess, 266.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
License Plate Detection: [257.34130859375, 59.17911148071289, 359.5411376953125, 98.21047973632812, 0.7680945992469788, 0.0]
results_final {'car': {'bbox': [257.34130859375, 59.17911148071289, 359.5411376953125, 98.21047973632812]}, 'license_plate': {'bbox': [257.34130859375, 59.17911148071289, 359.5411376953125, 98.21047973632812], 'text': 0, 'bbox_score': 0.7680945992469788, 'text_score': 0}}
License Plate Detection: [595.8501586914062, 97.13385009765625, 747.00537109375, 143.4988555908203, 0.7447417974472046, 0.

In [1]:
from ultralytics import YOLO
import cv2
from sort.sort import *
import numpy as np
from utils import Get_Car, Read_License_Plate, Write_CSV    
import pandas as pd

#cocomodels trained on coco dataset to detect vehicles
model = YOLO('yolov8n.pt')
LP_detector = YOLO('license_plate_detector.pt')


cap =cv2.VideoCapture('testUK3.mp4')
vehicles = [2,3,5,7]


frame_num =-1
ret = True
mot_tracker = Sort()
results = {}
while ret:
    frame_num += 1
    f = True
    ret, frame = cap.read()

    if ret:
        if frame_num >15:
            break
            
        results[frame_num]= {}
 #detect Vehicles
        detections = model(frame)[0]
        detections_ = []
       # print(detections)
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            #print(detection)
            if int(class_id) in vehicles:
                detections_.append([x1,y1,x2,y2, score])

#track vehicles from videos
        
        track_ids = mot_tracker.update(np.asarray(detections_))
        print("result track ids:", track_ids)
#Detect License Plates
        LP = LP_detector(frame)[0]
        for lp in LP.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = lp
            print("License Plate Detection:", lp)

        #assign license plate to vehicle
            #if track_ids.any():
            xcar1, ycar1, xcar2, ycar2, car_id = Get_Car(lp,track_ids)
            print(Get_Car(lp, track_ids))
        
            if car_id != -1: #crop License plate
                lp_crop = frame[int(y1): int(y2), int(x1):int(x2),:]

            #process License plate
                    
                lp_crop_gray = cv2.cvtColor(lp_crop, cv2.COLOR_BGR2GRAY)
                _, lp_crop_tsh = cv2.threshold(lp_crop_gray, 90, 255, cv2.THRESH_BINARY_INV )
                # cv2.imshow('lp_crop', lp_crop)
                # cv2.imshow('Threshold', lp_crop_tsh)
                # cv2.waitKey(0)
            
            # Read License Plate text
                lp_text, lp_text_score = Read_License_Plate(lp_crop_tsh)
                print(Read_License_Plate(lp_crop_tsh))

                if lp_text is not None:
                    #car_id = str(car_id.tolist()) 
                    results[frame_num][str(car_id)] = {
                        'car' : {'bbox':[xcar1, ycar1, xcar2, ycar2]},
                        'license_plate': {
                            'bbox': [x1, y1, x2, y2], 
                            'text': lp_text,
                            'bbox_score':score,
                            'text_score':lp_text_score
                                                }
                                            }                                                                
                                                                
                
                    print('results_final', results[frame_num][str(car_id)])
                                            
                    

           
            #Write results
Write_CSV(results, 'output.csv')


Using CPU. Note: This module is much faster with a GPU.



0: 384x640 2 cars, 388.0ms
Speed: 13.0ms preprocess, 388.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)
result track ids: [[     454.59      1.3412       597.1      206.83           2]
 [     208.87      1.3687       465.6      182.88           1]]

0: 384x640 3 license_plates, 272.0ms
Speed: 8.0ms preprocess, 272.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
License Plate Detection: [598.1221313476562, 96.8009033203125, 749.4193725585938, 144.1802215576172, 0.7610451579093933, 0.0]
(-1, -1, -1, -1, -1)
License Plate Detection: [267.4906921386719, 60.846439361572266, 360.46923828125, 97.29545593261719, 0.7333720922470093, 0.0]
[     208.87      1.3687       465.6      182.88           1]
(None, None)
License Plate Detection: [596.8270263671875, 0.0, 760.8195190429688, 95.8116226196289, 0.3948495090007782, 0.0]
(-1, -1, -1, -1, -1)

0: 384x640 2 cars, 1 truck, 329.0ms
Speed: 8.0ms preprocess, 329.0ms inference, 6.0ms postprocess per image at

KeyboardInterrupt: 

In [6]:
Write_CSV(results, 'output.csv')

{'car': {'bbox': [197.8491962500167, 0.5388852616178923, 459.31593928942334, 180.55369076928736]}, 'license_plate': {'bbox': [257.34130859375, 59.17911148071289, 359.5411376953125, 98.21047973632812], 'text': 'MA08BZM', 'bbox_score': 0.7680945992469788, 'text_score': 0.828133559985971}}
{'car': {'bbox': [450.39764404296875, 0.6888427734375, 768.0, 204.64697265625]}, 'license_plate': {'bbox': [595.8501586914062, 97.13385009765625, 747.00537109375, 143.4988555908203], 'text': 'LA56LHC', 'bbox_score': 0.7447417974472046, 'text_score': 0.17519877249081206}}
{'car': {'bbox': [446.0638960294906, 0.13651226947311557, 767.1881438444381, 204.86567865505293]}, 'license_plate': {'bbox': [591.368896484375, 96.91183471679688, 745.3617553710938, 144.06288146972656], 'text': 'LM56LHC', 'bbox_score': 0.7442050576210022, 'text_score': 0.3448537597921746}}
{'car': {'bbox': [143.66438046869135, -0.8815114284437016, 447.89352225740606, 180.08306241567425]}, 'license_plate': {'bbox': [246.926513671875, 58.

Write Results